# Preparing School Planning Areas data
## Code involves:
1. Extracting the required data from Arcgis online school planning areas map service,
2. Converting SPA polygons from Web Mercator into longitude and latitude format,
3. Creating two separae GeoDataFrames for the SPAs and the schools.
4. Spatially joining the schools to their respective SPAs,
5. Finally saving into new complete dataset.

#### *NOTE*: All column names and pathways will need to be updated to reflect how saved in your dataset.

In [ ]:
#Loading libraries and link to School Planning Areas map.
import requests
from shapely.geometry import shape, Polygon, Point
import pandas as pd
from pyproj import Transformer
import openpyxl as openpyxl
import geopandas as gpd
url = "https://services-eu1.arcgis.com/9HteQxumPOXiqlpG/arcgis/rest/services/School_Planning_Areas_2011_Map_Viewer/FeatureServer/0/query" #Link to SPA map

params = {
    'where': '1=1',
    'outFields': '*',
    'returnGeometry': 'true',
    'f': 'json'
} #Restricting what is scraped to just the json geometry data and attributes required

resp= requests.get(url, params=params)
data= resp.json()
transformer= Transformer.from_crs("epsg:3857", "epsg:4326", always_xy=True) #Converting co-ordinates from Web Mercator into latitude and longitude format

spaPolygon = []
spaID = []
spaName = []#preparing empty lists to hold needed data

for feature in data['features']:
    attr = feature['attributes'] #Extracts other information (e.g. name) to extract on each SPA
    geom = feature['geometry'] #Extracts The geographical data for each SPA
    rings = geom.get('rings', [])
    
    poly_3857 = Polygon(rings[0]) #creates polygon from Web Mercator co-ordinates
    poly_coords_3857 = list(poly_3857.exterior.coords) #List of all polygon co-ordinates in web mercator format
    poly_coords_4326 = [transformer.transform(x, y) for x, y in poly_coords_3857] #Transorming to long-lat format.
    poly_4326 = Polygon(poly_coords_4326) #Creating polygons in transformed long-lat format
    
    spaPolygon.append(poly_4326)#Assigning polygon list of co-ordinates to spaPolygon list
    spaID.append(attr.get("OBJECTID")) #Assigning spa nuerical codes to spaID list
    spaName.append(attr.get("PLANNING_AREA") or attr.get("Name") or "Unknown")#Assigning spa names to spaName list

spaGDF = gpd.GeoDataFrame({
    'SPA_ID': spaID,
    'SPA': spaName,
    'geometry': spaPolygon
}, crs="EPSG:4326") # Creating a GeoDataFrame for SPA polygons

dataOverall = pd.read_excel('C:\\') # Add path to overall schools dataset (this is combined schools data from Department of Education annual csv's)

schoolGDF = gpd.GeoDataFrame(
    dataOverall,
    geometry=gpd.points_from_xy(dataOverall.long, dataOverall.lat),
    crs="EPSG:4326"
) # Creating a second GeoDataFrame of schools points using (longitude, latitude)

schoolMatchSPA = gpd.sjoin(schoolGDF, spaGDF[['SPA_ID', 'SPA', 'geometry']], how='left', predicate='within') #Spatially joinng the 2 GeoDataFrames

print(schoolMatchSPA[['School', 'lat', 'long', 'SPA_ID', 'SPA']].head()) #Checking it worked #Change column names to reflect your dataset

In [ ]:
#Saving the dataset for analysis;
schoolMatchSPA.to_excel(".xlsx", index=False) #Add pathway to folder and name dataset

In [ ]:
#Visualising the SPA
import matplotlib.pyplot as plt
import geopandas as gpd
fig, ax = plt.subplots(figsize=(12, 10))
spaGDF.plot(
    ax=ax,
    column='SPA_ID',     
    cmap='tab20',        
    edgecolor='black',   
    linewidth=0.3,
    legend=False
)
plt.title("School Planning Areas", fontsize=16)
plt.axis('off')  
plt.savefig("spa_map_coloured.png", dpi=300, bbox_inches='tight') #Saves to folder
plt.show()